Analysis of the winding parameters of a 1250 kVA Superconducting Transformer
-------------------------------------------------------------------------------------------------------------------

The transformer and its parameters originally published in DOI:10.1109/TASC.2014.2345349.

Main parameters of the  transformer:

- $S_n$ = 1250 kVA - nominal power
- $f$ = 50 Hz - network frequency
- $\epsilon$ = 5.8 % - short circuit impedance
- $U_1 / U_2$ = 10.5 kV / 0.4 kV
- $I_1 / I_2$ = 69 A / 1804 A
- Connection group: YNyn0


In [1]:
import os
import sys
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from src.base_functions import turn_voltage,short_circuit_impedance

Calculating the turn voltage from the winding parameters:

In [2]:
# seconday - outer, primary inner winding
Np = 262.0 # turns in the primary winding
Up = 10500 # V 
utp = Up/Np/1.73
print("Turn voltage from primary winding data: ", round(utp,2), "V")

# secondary winding
Ns = 10
Us = 400
uts= Us/1.73/Ns
print("Turn voltage from secondary winding data:",round(uts,2), "V")

# BSSCO tape dimensions
w = 4.8  # mm
h = 0.32 # mm
A = 4.8*0.32 # 1.536 mm2

# Winding areas
a_lv = 13.5 * 342.5 # mm
a_hv = 8* 355

Nd = 22 # number of discs/pancakes
ff_lv = A*Np/a_lv*100
ff_hv = A*Ns*Nd/a_hv*100

print('Filling factor in LV (primary) winding', round(ff_lv,2), '%')
print('Filling factor in HV (secondary) winding',round(ff_hv,2), '%')

I1 = 69.
I2 = 1804.

ae_lv = (a_lv*(ff_lv/100))
ae_hv = (a_hv*(ff_hv/100))

print("Amperturns should balanced:", I1*Np, I2*Ns)

j_lv = I1*Np/ae_lv
j_hv = I2*Ns/ae_hv

print('j in LV:',round(j_lv,2), 'A/mm2', round(ae_lv,1), round(a_lv,1))
print('j in HV:',round(j_hv,2), 'A/mm2', round(ae_hv,1), round(a_hv,1))


Turn voltage from primary winding data:  23.17 V
Turn voltage from secondary winding data: 23.12 V
Filling factor in LV (primary) winding 8.7 %
Filling factor in HV (secondary) winding 11.9 %
Amperturns should balanced: 18078.0 18040.0
j in LV: 44.92 A/mm2 402.4 4623.8
j in HV: 53.39 A/mm2 337.9 2840


Independent design parameters for the two-winding transformer model:

- $r_c$ core radius : 155 mm
- $b_c$ flux density: 1.5 T
- $j_{in}$ current density in the inner winding: 44.92 $A/mm^2$ -- 100% filling factor assumed
- $j_{ou}$ current density in the outer winding: 53.39 $A/mm^2$
- $h_{in}$ the height of the inner winding: 342.5 mm
- $m_{gap}$ main gap - the distance between two main windings: 34.5 mm

Cooling System:

- Cryostat inner / outer diameter: 330/740 mm
- Operation temperature: 77 K


The core filling factor in the core's column was calculated from the other calculation of the turn voltage:

In [3]:
ut = turn_voltage(1.5, 155.0, 0.92, 50.0)
print('turn voltage:',round(ut,2), 'V')

sci = short_circuit_impedance(1250., 3, 50, 1.03, 23.1, 342.5, 0, 205, 13.5, 253, 8., 34.5)
print('SCI:',sci, '%')

turn voltage: 23.12 V
SCI: 5.29 %


The calculated short circuit impedance (5.29%) is smaller than the given data in the paper, but it can be acceptable. Because the model contains a special HV winding where the size of  the top and bottom pancake is differs from the other parts of the winding.

The goal of the following code calculates all of the parameters by the built-in two winding model of the transformer. All of the data of the LV and HV windings (inner/outer radius, losses, amperturns) calculated from the independent variables and  

In [4]:
from importlib_resources import files
from src.two_winding_model import TransformerDesign, TwoWindingModel

path = files("data").joinpath("1250kVA_sc_transformer.json")

import json

with open(path) as json_file:
    data = json.load(json_file)

transformer = TransformerDesign.from_dict(data)

trafo_model = TwoWindingModel(input=transformer)
trafo_model.calculate(is_sc=True)
print("LV winding:")
print(trafo_model.lv_winding)
print("HV winding:")
print(trafo_model.hv_winding)

LV winding:
WindingDesign(inner_radius=205.1, thickness=13.5, winding_height=342.5, filling_factor=8.7, current_density=44.92, mass=10.3, dc_loss=0.0, ac_loss=1663.0666171977766, outer_radius=218.6, cable_length=1266.23, amper_turns=18069.8)
HV winding:
WindingDesign(inner_radius=253.1, thickness=8.2, winding_height=356.2, filling_factor=11.6, current_density=53.45, mass=10.5, dc_loss=0.0, ac_loss=1695.3499918222296, outer_radius=261.3, cable_length=1290.81, amper_turns=18109.8)


In [5]:
# FEM calculation
trafo_model.fem_simulation()

Mesh Generator: Initial mesh generation
Mesh generator: Triangle
Mesh Generator: Reading initial mesh from memory
Solver: Solver - MUMPS (none)
Solver: Elapsed time: 00:00.756 s
Magnetic Energy 36.1038335428817
zb, ib: 88.2 ohm 68.73 A
SCI: 5.44 [%]
Bax [mT] = 63.88059481829217
Brad [mT] = 34.931126895950015
Values along the outer winding: [(0.029128001810690196, 0.034931126895950015), (0.050827730564536266, 0.013995471297917561), (0.05588725071682793, 0.008240268880776114), (0.05886954390963839, 0.005638255023196592), (0.06077329192829234, 0.004242084637681387), (0.06198176035801467, 0.003445644623909061), (0.06276385272892504, 0.002995500599112688), (0.06327437776653683, 0.002760552277617734), (0.06359944969665898, 0.002661568692436944), (0.06379442937349428, 0.0026641586971951625), (0.06388059481829217, 0.002728564552750612), (0.06386326031218509, 0.0028002328471558036), (0.06375556424426587, 0.0028587919261724437), (0.06355546873347745, 0.0028738248556079427), (0.06317272884807773,

Detailed calculation of the superconducting losses and validating the results in the comparison with the published papers:

In [8]:
from src.superconductor_losses import cryo_surface,cryostat_losses, magnusson_ac_loss, supra_winding_ac_loss
a_cs = cryo_surface(205.0, 370, 800)  # [mm2]
cryo_loss = cryostat_losses(a_cs)
#thermal_loss = thermal_incomes(self.input.required.lv.ph_current, self.input.required.hv.ph_current)

print('a_cs:',a_cs)
#print('cryostat losses:', cryo_loss)
print(cryo_loss)

# HV winding
print('hv:',magnusson_ac_loss(63.89*1e-3, 34.39*1e-3, 50, 69, Ic=115), 'W/m')
print('lv:',magnusson_ac_loss(63.89*1e-3, 34.39*1e-3, 50, 1804/20, Ic=115), ',W/m')

# Calculation with the averaged values of Bax and Brad along the winding
B = [(0.029128001810690196, 0.034931126895950015), (0.050827730564536266, 0.013995471297917561), (0.05588725071682793, 0.008240268880776114), (0.05886954390963839, 0.005638255023196592), (0.06077329192829234, 0.004242084637681387), (0.06198176035801467, 0.003445644623909061), (0.06276385272892504, 0.002995500599112688), (0.06327437776653683, 0.002760552277617734), (0.06359944969665898, 0.002661568692436944), (0.06379442937349428, 0.0026641586971951625), (0.06388059481829217, 0.002728564552750612), (0.06386326031218509, 0.0028002328471558036), (0.06375556424426587, 0.0028587919261724437), (0.06355546873347745, 0.0028738248556079427), (0.06317272884807773, 0.0028134675986741797), (0.06259672900939289, 0.0026311711550385253), (0.061686454242632804, 0.0022513791514020554), (0.06025066397296713, 0.0015527807458504128), (0.05795286910805797, 0.000685192255041045), (0.054315535780667834, 0.003759431911852573), (0.048564227979538355, 0.010423416210258551)]
print('hv average:',supra_winding_ac_loss(B, 50, 69, Ic=135), 'W/m')

a_cs: 3188323.844311941
0.029077513460124906
hv: 0.7448746890442886 W/m
lv: 0.7693234418649139 ,W/m
29.128001810690197 34.931126895950015 0.01864999814490323 0.6415251063070724
50.82773056453627 13.995471297917561 0.09338117736158892 0.6954714319502531
55.88725071682793 8.240268880776114 0.18166718997787898 0.7037897872187975
58.86954390963839 5.638255023196592 0.2779429894040921 0.7057629943657476
60.77329192829234 4.242084637681387 0.37931907234553014 0.7064154393870273
61.98176035801467 3.445644623909061 0.4839327327133864 0.70670348265898
62.76385272892504 2.995500599112688 0.5906416771144508 0.7068691309623825
63.27437776653683 2.760552277617734 0.6987183563249161 0.7069889887729428
63.59944969665898 2.661568692436944 0.8076659275810821 0.7070926888135985
63.794429373494275 2.6641586971951625 0.917135863993678 0.7071967897271394
63.88059481829217 2.728564552750612 1.0268366440858883 0.7073112350020259
63.86326031218509 2.8002328471558036 1.1364909837380421 0.7074380687349637
63.75

In [10]:
print("Values without average:")
r_m = 244
length = Np * 2*r_m*3.14 * 1e-3 
print(length*3)

length_sec = 10*22*2*3.14*1e-3*290
loss_sec = 0.744 *length_sec*3
print(length_sec*3, loss_sec)

print("Loss calculation for 21 segments in the hv winding:")
print('P_sec', 0.14198*length_sec*3 )
print('P_pri', 0.14*length*3)




Values without average:
1204.40352
1201.9920000000002 894.282048
Loss calculation for 21 segments in the hv winding:
P_sec 170.65882416
P_pri 168.61649280000003
